# Modèles multimodaux - CLIP

## README
Ce notebook permet la création et l'évaluation d'un modèle basé sur l'architecture existatne CLIP.

## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import os
import time
import pickle
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from src.models.multimodal_clip import MultiModalCLIPBasedClassifier
from src.models.model_wrappers import ModelWrapperFactory

## 2. Chargement des données

In [ ]:
documents = pd.read_parquet(PATHS.metadata / "df_data_sets.parquet")
data_sets = pd.read_parquet(PATHS.metadata / "df_data_sets.parquet")
labels = pd.read_parquet(PATHS.metadata / "df_encoded_labels.parquet")

In [ ]:
documents.shape, data_sets.shape, labels.shape

In [ ]:
X_train = documents[data_sets.data_set == "train"].index
y_train = labels[data_sets.data_set == "train"].label

X_val = documents[data_sets.data_set == "val"].index
y_val = labels[data_sets.data_set == "val"].label

X_test = documents[data_sets.data_set == "test"].index
y_test = labels[data_sets.data_set == "test"].label

In [ ]:
model = MultiModalCLIPBasedClassifier()

# 3. Apprentissage

In [ ]:
# environ 3'30'' sur 400k images
t0 = time.time()
model.fit(X_train, y_train)
print(f"Terminé en {time.time()-t0:.2f} secondes")

In [ ]:
path = PATHS.models / "mmo_clip_logistic_regressor.joblib"
model.save(path)

In [ ]:
name = "CLIP-based Logistic Regressor"
wrapper = ModelWrapperFactory.make_mmo_clip_wrapper(name, path)
wrapper.visual_report()